In [1]:
%matplotlib widget
%load_ext nb_black

import numpy as np
import os
import pandas
import pytest
import sys

from ase import io
from ase.visualize import view
import nglview as ngl

sys.path.append("../")
from graphene_analysis import analysis

import matplotlib.pyplot as plt

/Users/Fabsilein/.pyenv/versions/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


<IPython.core.display.Javascript object>

In [4]:
path = "../tests/files/trajectories/pristine/"

simulation = analysis.Simulation(path, "Pristine")
simulation.read_in_simulation_data()
simulation.set_sampling_times(
    start_time=0, end_time=-1, frame_frequency=1, time_between_frames=100
)

simulation.find_defective_atoms()
# simulation.find_atoms_around_defects_within_cutoff()

Using the topology from ../tests/files/trajectories/pristine/npt.pdb.
Creating universes for ../tests/files/trajectories/pristine/npt.dcd.
SUCCESS: New sampling times.
Start time: 	 	0 	 fs
End time: 	 	1700 	 fs
Time between frames: 	100 	 fs
Frame frequency: 	1


KeyNotFound: ('Cannot guess defect type from system name given: Pristine', 'Please rename the system.')

<IPython.core.display.Javascript object>

In [3]:
invariant_positions = simulation.compute_local_environments_geometry()

  0%|          | 0/17 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [4]:
fig, ax = plt.subplots(1, 1)

for count, defect in enumerate(invariant_positions):

    ax.scatter(defect[:, 0], defect[:, 1])

# ax.scatter(invariant_positions[10][:,0],invariant_positions[10][:,1])


ax.set_aspect("equal")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<IPython.core.display.Javascript object>

In [38]:
from mpl_toolkits import mplot3d
import scipy

X, Y = np.meshgrid(np.arange(-4.0, 4.0, 0.1), np.arange(-4.0, 4.0, 0.1))
XX = X.flatten()
YY = Y.flatten()

data = defect
A = np.c_[
    np.ones(data.shape[0]), data[:, :2], np.prod(data[:, :2], axis=1), data[:, :2] ** 2
]
C, res, _, _ = scipy.linalg.lstsq(A, data[:, 2])

# evaluate it on a grid
Z = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX * YY, XX ** 2, YY ** 2], C).reshape(
    X.shape
)
print(C)

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
ax.scatter(data[:, 0], data[:, 1], data[:, 2])



[ 0.0826877  -0.11933532  0.06694045  0.03864141 -0.04426316  0.0404186 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<IPython.core.display.Javascript object>

In [43]:

def function(x_dat,y_dat, coeff):
    return coeff[0]+coeff[1]*x_dat+coeff[2]*y_dat+coeff[3]*x_dat*y_dat+ coeff[4]*x_dat**2+ coeff[5]*y_dat**2 
    
x_dat = data[:, 0]
y_dat = data[:, 1]

resid = np.sum(np.square(function(x_dat,y_dat,C)-data[:,2]))


<IPython.core.display.Javascript object>

In [46]:
r2 = 1 - resid / (data[:, 2].size * data[:, 2].var())
r2

0.9735346316038488

<IPython.core.display.Javascript object>

In [44]:
data[:, 2].var()

0.201913123793185

<IPython.core.display.Javascript object>

In [16]:
view(simulation.topology[simulation.defective_atoms_ids_clustered[3]])

<Popen: returncode: None args: ['/Users/Fabsilein/.pyenv/versions/3.9.1/bin/...>

In [12]:
simulation.orientations_per_defect

array([1.57079633, 1.57079633, 0.52363049, 0.52363049, 1.57079633,
       1.57079633, 1.57079633, 0.52352867, 1.57079633, 0.52357941,
       0.52371815, 0.52352848, 1.57079633, 1.57079633, 0.52352867,
       0.52357943, 0.52366729, 0.52361655, 1.57079633, 0.52357931,
       0.52375531, 1.57079633, 0.52352875, 0.52362982, 1.57079633,
       0.52352901, 0.52380626, 0.52363049, 0.52366753, 0.52352881,
       0.52357868, 0.52352833, 1.57079633, 1.57079633, 0.52352901,
       0.52352833])

In [15]:
view(simulation.topology[simulation.defective_atoms_ids_clustered[2]])

<Popen: returncode: None args: ['/Users/Fabsilein/.pyenv/versions/3.9.1/bin/...>

In [8]:
mean = np.array([0.0,0.0,0.0])
cov = np.array([[1.0,-0.5,0.8], [-0.5,1.1,0.0], [0.8,0.0,1.0]])
data = np.random.multivariate_normal(mean, cov, 50)

In [14]:
data

array([[-0.24268258,  0.99001868,  0.40041638],
       [-1.58303711, -0.25538717, -1.40148475],
       [-0.4245625 ,  0.14765144, -0.60625481],
       [ 0.17409913, -0.70621061, -0.55429747],
       [ 0.1466981 , -0.08355283,  0.10063218],
       [ 0.15668731, -0.50150027, -0.04464239],
       [-0.03369849, -0.05272379,  0.82097331],
       [-0.3483041 , -0.67165415,  0.10525948],
       [ 0.42155432,  0.05211819,  0.4953925 ],
       [-0.00843094,  2.0111955 ,  0.64736024],
       [-0.47985917, -0.73971545, -1.07360847],
       [-0.08246537,  1.55010696,  0.54611603],
       [ 0.53808137, -0.13532633, -0.11750471],
       [-1.19096098,  1.73570675,  0.06910027],
       [-1.33595383,  0.91050814, -1.29136823],
       [ 0.38982025,  1.17435341,  0.15564817],
       [ 0.11413607,  0.43073221,  0.59263044],
       [ 0.56999005, -0.05528372,  1.29804901],
       [-1.03567462,  0.65389149, -0.86485506],
       [ 0.40508937, -1.53918891,  0.20734666],
       [-0.21620066,  0.1391652 , -0.662

In [16]:
A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2]
A
A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
A

array([[-0.24268258,  0.99001868,  1.        ],
       [-1.58303711, -0.25538717,  1.        ],
       [-0.4245625 ,  0.14765144,  1.        ],
       [ 0.17409913, -0.70621061,  1.        ],
       [ 0.1466981 , -0.08355283,  1.        ],
       [ 0.15668731, -0.50150027,  1.        ],
       [-0.03369849, -0.05272379,  1.        ],
       [-0.3483041 , -0.67165415,  1.        ],
       [ 0.42155432,  0.05211819,  1.        ],
       [-0.00843094,  2.0111955 ,  1.        ],
       [-0.47985917, -0.73971545,  1.        ],
       [-0.08246537,  1.55010696,  1.        ],
       [ 0.53808137, -0.13532633,  1.        ],
       [-1.19096098,  1.73570675,  1.        ],
       [-1.33595383,  0.91050814,  1.        ],
       [ 0.38982025,  1.17435341,  1.        ],
       [ 0.11413607,  0.43073221,  1.        ],
       [ 0.56999005, -0.05528372,  1.        ],
       [-1.03567462,  0.65389149,  1.        ],
       [ 0.40508937, -1.53918891,  1.        ],
       [-0.21620066,  0.1391652 ,  1.   